In [ ]:
# Configuration Constants
import os
from openai import OpenAI, AzureOpenAI
from botocore.exceptions import ClientError
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Base URLs from environment
BASE_URL = os.getenv("GOOP_API_URL", "http://localhost:8080")
API_KEY = os.getenv("GOOP_API_KEY", "test")

OPENAI_ENDPOINT = f"{BASE_URL}/openai/v1"
OPENAI_PROXY_ENDPOINT = f"{BASE_URL}/openai-proxy/v1"
AZURE_ENDPOINT = f"{BASE_URL}/azure"
BEDROCK_ENDPOINT = f"{BASE_URL}/bedrock"
GEMINI_ENDPOINT = f"{BASE_URL}/gemini"

# Environment Variables
GEMINI_PROJECT_ID = "encrypted-llm"

# Models
OLLAMA_MODEL = "llama3.2:latest"
BEDROCK_SONNET_4 = "us.anthropic.claude-sonnet-4-20250514-v1:0"
GEMINI_FLASH_LITE = "gemini-2.5-flash-lite"
OPENAI_GPT4O_MINI = "gpt-4o-mini"

print(f"Configuration loaded successfully!")
print(f"Base URL: {BASE_URL}")
print(f"API Key: {'***' + API_KEY[-4:] if len(API_KEY) > 4 else '***'}")

# Base OpenAI Client

```python
from openai import OpenAI, AzureOpenAI
client = OpenAI(
    base_url="http://localhost:8080/openai/v1",
    api_key="test",
)
```

In [ ]:
#!/usr/bin/env python3

client = OpenAI(
    base_url=OPENAI_ENDPOINT,
    api_key=API_KEY,
)

print(client.models.list())

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Say this is a test",
        }
    ],
    model=f"gpt-4o-mini",
    stream=False,
)

print(chat_completion.choices[0].message.content)

In [ ]:
### Streaming
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Write me a pretty short story",
        }
    ],
    model="gpt-4o-mini",
    stream=True,
)

for i in chat_completion:
    print(i.choices[0].delta.content, end="")
print()

In [ ]:
### Streaming with OPenai Proxy
client = OpenAI(
    base_url=OPENAI_PROXY_ENDPOINT,
    api_key=API_KEY,
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Write me a pretty short story",
        }
    ],
    model=f"gemini/{GEMINI_FLASH_LITE}",
    stream=True,
)

for i in chat_completion:
    print(i.choices[0].delta.content, end="")
print()

### Boto3 Bedrock Runtime Client

```python
def _replace_headers(request: AWSRequest, **kwargs):
    request.headers = {"Authorization": "Bearer test"}

client = boto3.client("bedrock-runtime", endpoint_url="http://localhost:8080/bedrock")
client.meta.events.register("before-send.bedrock-runtime.*", _replace_headers)
```

In [ ]:
import boto3
from botocore.exceptions import ClientError


def _replace_headers(request, **kwargs):
    request.headers = {"Authorization": "Bearer " + API_KEY}


bedrockClient = boto3.client(
    "bedrock-runtime", endpoint_url=f"{BASE_URL}/bedrock", region_name="us-east-1"
)
bedrockClient.meta.events.register("before-send.bedrock-runtime.*", _replace_headers)

model_id = BEDROCK_SONNET_4  # "us.anthropic.claude-sonnet-4-20250514-v1:0"
user_message = "Say 'hi'"
conversation = [
    {
        "role": "user",
        "content": [{"text": user_message}],
    }
]

try:
    # Send the message to the model, using a basic inference configuration.
    response = bedrockClient.converse(
        modelId=model_id,
        messages=conversation,
        inferenceConfig={"maxTokens": 512, "temperature": 0.5, "topP": 0.9},
    )
    response_text = response["output"]["message"]["content"][0]["text"]
    print(response_text)

except (ClientError, Exception) as e:
    print(f"ERROR: Can't invoke '{model_id}'. Reason: {e}")
    exit(1)

### Google Vertex Client

```python
import vertexai
from vertexai.preview.generative_models import GenerativeModel

vertexai.init(
    project=os.getenv("VERTEX_AI_PROJECT_ID"),
    api_endpoint="http://localhost:8080/vertex",
)

generative_multimodal_model = GenerativeModel("gemini-1.5-flash-002")
response = generative_multimodal_model.generate_content(["Say hi"])

print(response)
```

In [ ]:
# To run this code you need to install the following dependencies:
# pip install google-genai

import base64
import os
from google import genai
from google.genai import types


genai_client = genai.Client(
    http_options=types.HttpOptions(
        base_url=GEMINI_ENDPOINT, headers={"Authorization": f"Bearer {API_KEY}"}
    ),
    api_key=API_KEY,
)

model = "gemini-2.5-flash-lite"
contents = [
    types.Content(
        role="user",
        parts=[
            types.Part.from_text(text="""Say hi and then give me a story about a cat"""),
        ],
    ),
]
tools = [
    types.Tool(googleSearch=types.GoogleSearch()),
]
generate_content_config = types.GenerateContentConfig(
    thinking_config=types.ThinkingConfig(
        thinking_budget=0,
    ),
    tools=tools,
)

for chunk in genai_client.models.generate_content_stream(
    model=model,
    contents=contents,
    config=generate_content_config,
):
    print(chunk.text, end="")

### ELL Framework Example

In [ ]:
import ell
from pydantic import Field
import requests
from bs4 import BeautifulSoup

openai_client = OpenAI(
    base_url=OPENAI_PROXY_ENDPOINT,
    api_key=API_KEY,
)

ell.init(verbose=True)
    

"""
BEDROCK CLIENT
"""


@ell.simple(model=f"gemini/{GEMINI_FLASH_LITE}", client=openai_client)
def bedrock_chat(prompt: str) -> str:
    return prompt


print(f'Bedrock \n\n {bedrock_chat("Hello, how are you?")}\n\n')

In [ ]:
"""
OpenAI Client
"""

@ell.simple(model="gpt-4o-mini", max_tokens=10, client=client)
def openai_client(prompt: str) -> str:
    return prompt


print(f'OpenAI \n\n {openai_client("Hello, how are you?")}\n\n')

In [ ]:
"""
TOOL USAGE
"""

@ell.tool()
def get_html_content(
    url: str = Field(
        description="The URL to get the HTML content of. Never include the protocol (like http:// or https://)"
    ),
):
    """Get the HTML content of a URL."""
    response = requests.get("https://" + url)
    soup = BeautifulSoup(response.text, "html.parser")
    return soup.get_text()[:100]


# OpenAI Client
@ell.complex(
    model="gpt-4o-mini",
    tools=[get_html_content],
    client=client,
)
def openai_get_website_content(website: str) -> str:
    return f"Tell me what's on {website}"


print("OpenAI Client Tool Use\n\n")
output = openai_get_website_content("new york times front page")
if output.tool_calls:
    print(output.tool_calls[0]())


# Bedrock Client
@ell.complex(
    model="anthropic.claude-3-haiku-20240307-v1:0",
    tools=[get_html_content],
    client=bedrockClient,
)
def bedrock_get_website_content(website: str) -> str:
    """You are an agent that can summarize the contents of a website."""
    return f"Tell me what's on {website}"


print("\n\nBedrock Client Tool Use\n\n")
output = bedrock_get_website_content("new york times front page")
if output.tool_calls:
    print(output.tool_calls[0]())

In [ ]:
from openai import OpenAI, AzureOpenAI

client = OpenAI(
    base_url=OPENAI_PROXY_ENDPOINT,
    api_key=API_KEY,
)
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Whats up dog?",
        }
    ],
    model="bedrock/us.anthropic.claude-3-haiku-20240307-v1:0",
    stream=False,
)

print(chat_completion)
print(chat_completion.choices[0].message.content)

In [ ]:
from openai import OpenAI, AzureOpenAI

client = OpenAI(
    base_url=OPENAI_PROXY_ENDPOINT,
    api_key=API_KEY,
)
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Write me a pretty short story",
        }
    ],
    model=f"bedrock/{BEDROCK_SONNET_4}",
    stream=True,
)

# for i in chat_completion:
#     print(i)
# print()


for i in chat_completion:
    print(i.choices[0].delta.content, end="")
print()

In [ ]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_delivery_date",
            "description": "Get the delivery date for a customer's order. Call this whenever you need to know the delivery date, for example when a customer asks 'Where is my package'",
            "parameters": {
                "type": "object",
                "properties": {
                    "order_id": {
                        "type": "string",
                        "description": "The customer's order ID.",
                    },
                },
                "required": ["order_id"],
                "additionalProperties": False,
            },
        },
    }
]

messages = [
    {"role": "user", "content": "Hi, can you tell me the delivery date for my order? Its order id 12345."},
]


chat_completion = client.chat.completions.create(
    messages=messages,
    model=f"bedrock/{BEDROCK_SONNET_4}",
    stream=False,
    tools=tools,
    tool_choice="required",
)
print(chat_completion.choices[0])

In [ ]:
print(client.models.list())